# Exploring 33 records from 1543 (Kickback from Lex's R Code)
***
***DAEN690***

***George Mason University***

***Author:*** Grace Cox (Team LEGO)

***Date:*** November 15, 2021
***
`This Jupyter Notebook explores 33 records (of the 1543 OTHER records) that were returned after being ran through Lex's R Code. The process for exploring these records is as follows:`

1. Explore the 'DATASET' field, which indicates what type of Origin Location Reference appears within the 'REMARKS' field for each record (e.g., AIRPORT, NAVAID, AIRPORT DIRECTLY OVER, NAVAID DIRECTLY OVER, RUNWAY, FIX RADIAL, HELIPORT, etc) 
<br>

2. For each of the unique Origin Location References that appear in the 'DATASET' field:

    a. Run the uas_loc_parts() function of the dataframe 
    
    b. Link to the appropriate Origin Location Reference linking function (e.g., link_airportsCleaned_ident(), link_airportsCleaned_icao(), nav_link())
        i. Separate the field that were successfully linked with the Origin Location Reference Dataset(s) and those that were not. Those that were not linked successfully will be moved to the exceptions file for further review.
    c. Run the successfully linked dataframe through the uas_lat_long() function to calculate the UAS Latitude/Longitude coordinates for each respective record/remark. 

***
### INPUT: 
*Dataframe (read from a .csv file) containing 33 records, of 1543 records that did not hit on the initial 'XX NM dir XXX' regular expressions, and was not initially picked up by our Runways regular expressions.*

### OUTPUT:
<font color='red'>**EXCPETIONS FILE**</font>

*General Exceptions: 8 records (these remarks contain to valid UAS location information)*

*City: 1 record*

*CEDAR Airport: 1 record (no link to Airports)*


<font color='green'>**UAS LATITUDE/LONGITUDE COORDINATE CALCULATED**</font>

*FAA Airport Identifier: 22 records*

*NAVAID System: 1 record*


## Import Statements

In [1]:
# Import Statements
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import chart_studio.plotly as py
import re
import geopy
from geopy.distance import geodesic

from IPython.display import display, HTML

In [2]:
output33 = pd.read_csv('C:/Users/grace/OneDrive/Desktop/GMU/DAEN690/Grace - All_Points_Standard_33OUTOF415.csv')
output33

,index,REMARKS,UAS_Location_Runways,dateonly,timeonly,DATE,POD,PRIMARYCODE,SECONDARYCODES,REPORTINGFACILITY,...,UASDIM.CEDAR,UASFWROTOR.CEDAR,PILOTREPORTEDNMAC.CEDAR,TCASRA.CEDAR,SUMMARY.CEDAR,QAFINDINGS.CEDAR,RWYLOCATION,UASLOCATION,DATASET,cityname
0,1588,Aircraft reported a white uas 200 feet of left...,UNKN,2/17/2018,3:50,2018-02-17T03:50Z,DEN,UAS,NaN,DFW,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6 RBD,AIRPORT,NaN
1,1604,Aircraft reported a black uas 500 FT below Air...,UNKN,2/25/2018,5:18,2018-02-25T05:18Z,DEN,UAS,NaN,GSO,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,32NE GSO,AIRPORT,NaN
2,1774,Aircraft reported a grey quad copter with ligh...,UNKN,7/1/2018,2:36,2018-07-01T02:36Z,DEN,UAS,NaN,I90,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5 NM SE of IAH,AIRPORT,NaN
3,69,"Aircraft observed a quadcopter type uas, 200 f...",UNKN,7/10/2018,2:44,2018-07-10T02:44Z,DEN,UAS,NaN,I90,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10 NM N,CITY,Houston
4,1861,Aircraft reported a white quad copter passed w...,UNKN,9/15/2018,21:00,2018-09-15T21:00Z,DEN,UAS,PAGED,SLC,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1700 COR,AIRPORT,NaN
5,1971,Aircraft reported a fix wing uas or RC aircraf...,UNKN,3/27/2019,0:02,2019-03-27T00:02Z,DEN,UAS,NaN,DEN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1 NM N of APA,AIRPORT,NaN
6,1991,"Aircraft reported a uas 1,000 feet low while S...",UNKN,4/18/2019,4:44,2019-04-18T04:44Z,DEN,UAS,NaN,ZMA,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8 NM W of TRV,NAVAID,NaN
7,2037,Aircraft reported a blue uas 1 meter in diamet...,UNKN,6/1/2019,4:46,2019-06-01T04:46Z,DEN,UAS,NaN,CID,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,13NE CID,AIRPORT,NaN
8,2058,"Aircraft reported a uas at 2,400 feet while Nb...",UNKN,6/4/2019,2:00,2019-06-04T02:00Z,DEN,UAS,NaN,PHTO,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3 NM S of PHT,AIRPORT,NaN
9,2084,Aircraft reported a green and white uas 700 fe...,UNKN,6/22/2019,20:08,2019-06-22T20:08Z,DEN,UAS,NaN,MSY,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,31W MSY,AIRPORT,NaN


`Grab 8 records that were missing information or did not have any relevant information regarding UAS location to send back to Lex. These will either go into the exceptions file or use other relevant columns to obtain necessary information`

In [3]:
ef8 = output33.iloc[[0,4,15,22,24,27,30,31]]
ef8

,index,REMARKS,UAS_Location_Runways,dateonly,timeonly,DATE,POD,PRIMARYCODE,SECONDARYCODES,REPORTINGFACILITY,...,UASDIM.CEDAR,UASFWROTOR.CEDAR,PILOTREPORTEDNMAC.CEDAR,TCASRA.CEDAR,SUMMARY.CEDAR,QAFINDINGS.CEDAR,RWYLOCATION,UASLOCATION,DATASET,cityname
0,1588,Aircraft reported a white uas 200 feet of left...,UNKN,2/17/2018,3:50,2018-02-17T03:50Z,DEN,UAS,NaN,DFW,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6 RBD,AIRPORT,NaN
4,1861,Aircraft reported a white quad copter passed w...,UNKN,9/15/2018,21:00,2018-09-15T21:00Z,DEN,UAS,PAGED,SLC,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1700 COR,AIRPORT,NaN
15,655,Aircraft observed a uas off the left side whil...,UNKN,7/12/2020,21:20,2020-07-12T21:20Z,DEN,UAS,NaN,ZHU,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,090 of VCT,AIRPORT,NaN
22,796,"TOI #1 FDC NOTAM 0/0367, Aircraft observed a g...",UNKN,9/20/2020,22:00,2020-09-20T22:00Z,DEN,UAS,TOI-UAS-TFR,AGC,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1 FDC,AIRPORT,NaN
24,1052,Aircraft observed a small uas to the W while h...,UNKN,3/25/2021,23:44,2021-03-25T23:44Z,DEN,UAS,MOR,LAS,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2354 LAS,AIRPORT,NaN
27,1485,Aircraft observed a quad copter uas directly i...,UNKN,7/18/2021,21:43,2021-07-18T21:43Z,DEN,UAS,NaN,ISP,...,UAS Dimensions (feet): NA,Rotor Craft,No,No,FFT2862 CHECKED IN ON 7 MILE FINAL FOR RWY 24....,"Facility summary, weather and voice recording ...",NaN,7 MIL,AIRPORT - CEDAR,NaN
30,1512,Aircraft observed a uas off the 12 o'clock whi...,UNKN,8/4/2021,18:18,2021-08-04T18:18Z,DEN,UAS,NOTIFICATION-LATE,CXY,...,UAS Dimensions (feet): LARGE RC AIRPLANE,NaN,No,No,HELICOPTER IN THE TOWER PATTERN RY30 REPORTED ...,Facility summary and weather reviewed. A helic...,NaN,911 CAL,AIRPORT - CEDAR,NaN
31,1527,Aircraft observed a white uas off the right si...,UNKN,8/8/2021,20:31,2021-08-08T20:31Z,DEN,UAS,NaN,C90,...,UAS Dimensions (feet): 3 FEET,Rotor Craft,No,No,VOI942 WAS 15 MILES SOUTHEAST OF MDW HEADING 0...,QA reviewed. No NMAC filed.,NaN,15 MIL,AIRPORT - CEDAR,NaN


In [4]:
# Export to .csv 
ef8.to_csv('to_Lex_Standard_08OF33.csv', index = False)

`Drop the above 8 records from the 33 sent from Lex`

In [5]:
output25 = output33.drop([0,4,15,22,24,27,30,31]).reset_index(drop=True)
#output25

In [6]:
output25_df = output25[['REMARKS', 'DATASET', 'UASLOCATION', 'cityname']]
output25_df

,REMARKS,DATASET,UASLOCATION,cityname
0,Aircraft reported a black uas 500 FT below Air...,AIRPORT,32NE GSO,NaN
1,Aircraft reported a grey quad copter with ligh...,AIRPORT,5 NM SE of IAH,NaN
2,"Aircraft observed a quadcopter type uas, 200 f...",CITY,10 NM N,Houston
3,Aircraft reported a fix wing uas or RC aircraf...,AIRPORT,1 NM N of APA,NaN
4,"Aircraft reported a uas 1,000 feet low while S...",NAVAID,8 NM W of TRV,NaN
5,Aircraft reported a blue uas 1 meter in diamet...,AIRPORT,13NE CID,NaN
6,"Aircraft reported a uas at 2,400 feet while Nb...",AIRPORT,3 NM S of PHT,NaN
7,Aircraft reported a green and white uas 700 fe...,AIRPORT,31W MSY,NaN
8,Aircraft observed a Quad-copter uas while Sbou...,AIRPORT,1 NM NW of LAS,NaN
9,Aircraft observed a uas at the 6 o'clock posit...,AIRPORT,31 NM NNE of ATL,NaN


In [7]:
output25_df['DATASET'].unique()

array(['AIRPORT', 'CITY', 'NAVAID', 'AIRPORT - CEDAR'], dtype=object)

# AIRPORTS

In [8]:
output25_airport = output25_df[output25_df['DATASET'] == 'AIRPORT'].drop(columns = ['cityname']).reset_index(drop=True)
output25_airport

,REMARKS,DATASET,UASLOCATION
0,Aircraft reported a black uas 500 FT below Air...,AIRPORT,32NE GSO
1,Aircraft reported a grey quad copter with ligh...,AIRPORT,5 NM SE of IAH
2,Aircraft reported a fix wing uas or RC aircraf...,AIRPORT,1 NM N of APA
3,Aircraft reported a blue uas 1 meter in diamet...,AIRPORT,13NE CID
4,"Aircraft reported a uas at 2,400 feet while Nb...",AIRPORT,3 NM S of PHT
5,Aircraft reported a green and white uas 700 fe...,AIRPORT,31W MSY
6,Aircraft observed a Quad-copter uas while Sbou...,AIRPORT,1 NM NW of LAS
7,Aircraft observed a uas at the 6 o'clock posit...,AIRPORT,31 NM NNE of ATL
8,"MOR reported. Aircraft observed a 3 foot wide,...",AIRPORT,5 NM NW of LAS
9,Aircraft observed a uas operation 0.5 NM W of ...,AIRPORT,0.5 NM W of TPA


In [9]:
uasloc = output25_airport['UASLOCATION']

for i in range(len(output25_airport)):
    uasloc[i] = re.sub('(\d+(\.\d+)?)', r' \1 ', uasloc[i]).strip()

In [10]:
def uas_loc_parts(dataframe):
    '''
    This function takes the 'UASLOCATION' column from Lex's 'Kickbacks' and breaks it into the different parts of the 
    UAS Location, to include: Distance (NM), Direction Abbreviation, Airport Identifier
    
    input : dataframe is a df that includes the 'UASLOCATION' field
    output : the dataframe provided as input to the user with additional fields to include: 'Distance_NM', 'Bearing', 
                                                                                            'IDENT'
    '''
    uasLoc_col = dataframe['UASLOCATION']
    dist_list = []
    bearing_list = []
    ident_list = []
    dist_kilo = []
    bearDegree = []
    
    # Dictionary of Bearing Abbreviation to Degrees Conversions
    bearing_deg = {
    'N' : 0,
    'NNE' : 23,
    'NE' : 45,
    'ENE' : 68,
    'E' : 90,
    'ESE' : 113,
    'SE' : 135,
    'SSE' : 158,
    'S' : 180,
    'SSW' : 203,
    'SW' : 225,
    'WSW' : 248,
    'W' : 270,
    'WNW' : 293,
    'NW' : 315,
    'NNW' : 338
    }
    
    beardeg_list = list(bearing_deg.keys())
    
    for i in range(len(dataframe)):
        # Get Distance (NM) from UAS LOCATION and Convert Distance to Kilometers
        distNM = re.sub("[^\d\.]","", str(re.findall(r"[-+]?\d*\.\d+|\d+", uasLoc_col[i].split()[0])))
        if distNM == '':
            dist_list.append(0)
            dist_kilo.append(0.00)
        else:
            dist_list.append(distNM)
            dist_kilo.append(float(distNM) * 1.852)
        
        # Get IDENT from UASLOCATION
        ident = uasLoc_col[i].split()[-1].replace('of', '')
        ident_list.append(ident)
        
        # Get Bearing from UASLOCATION and Convert Bearing to Degrees
        #bearing = uasLoc_col[i].replace('NM', '').replace('of','').split()[-2]
        bearing = re.sub('(NM|of|OF)', ' ', uasLoc_col[i]).split()[-2]
        bearing_list.append(bearing)
        if bearing.isdigit():
            bearDegree.append(bearing)
        else:
            bearDegree.append(bearing_deg.get(bearing))
        
    # Append new fields to dataframe provided as input    
    dataframe['Distance_NM'] = dist_list
    dataframe['Distance_Kilometers'] = dist_kilo
    dataframe['Bearing'] = bearing_list
    dataframe['Bearing_Degrees'] = bearDegree
    dataframe['IDENT'] = ident_list
    
    
    return dataframe

In [11]:
all_parts22 = uas_loc_parts(output25_airport)
all_parts22

,REMARKS,DATASET,UASLOCATION,Distance_NM,Distance_Kilometers,Bearing,Bearing_Degrees,IDENT
0,Aircraft reported a black uas 500 FT below Air...,AIRPORT,32 NE GSO,32,59.264,NE,45,GSO
1,Aircraft reported a grey quad copter with ligh...,AIRPORT,5 NM SE of IAH,5,9.260,SE,135,IAH
2,Aircraft reported a fix wing uas or RC aircraf...,AIRPORT,1 NM N of APA,1,1.852,N,0,APA
3,Aircraft reported a blue uas 1 meter in diamet...,AIRPORT,13 NE CID,13,24.076,NE,45,CID
4,"Aircraft reported a uas at 2,400 feet while Nb...",AIRPORT,3 NM S of PHT,3,5.556,S,180,PHT
5,Aircraft reported a green and white uas 700 fe...,AIRPORT,31 W MSY,31,57.412,W,270,MSY
6,Aircraft observed a Quad-copter uas while Sbou...,AIRPORT,1 NM NW of LAS,1,1.852,NW,315,LAS
7,Aircraft observed a uas at the 6 o'clock posit...,AIRPORT,31 NM NNE of ATL,31,57.412,NNE,23,ATL
8,"MOR reported. Aircraft observed a 3 foot wide,...",AIRPORT,5 NM NW of LAS,5,9.260,NW,315,LAS
9,Aircraft observed a uas operation 0.5 NM W of ...,AIRPORT,0.5 NM W of TPA,0.5,0.926,W,270,TPA


In [12]:
def link_airportsCleaned(dataframe):
    '''
    This function links the provided dataframe to the airports_cleaned.csv file.
    
    input : dataframe is a df with an 'IDENT' field on which we will link to the airports_cleaned.csv
    output : the dataframe provided as input with airport lat/long fields appended
    '''
    
    # Read in Cleaned Airports dataset
    airportsC = pd.read_csv('C:/Users/grace/OneDrive/Desktop/GMU/DAEN690/airports_cleaned.csv')
    
    return pd.merge(dataframe, airportsC, on='IDENT', how='left')

In [13]:
air22_link = link_airportsCleaned(all_parts22)
air22_link

,REMARKS,DATASET,UASLOCATION,Distance_NM,Distance_Kilometers,Bearing,Bearing_Degrees,IDENT,GLOBAL_ID,NAME,LATITUDE,LONGITUDE,ICAO_ID
0,Aircraft reported a black uas 500 FT below Air...,AIRPORT,32 NE GSO,32,59.264,NE,45,GSO,4DACDC1C-0729-4FD5-AFA4-213599F5BF4B,Piedmont Triad Intl,36.101327,-79.941123,KGSO
1,Aircraft reported a grey quad copter with ligh...,AIRPORT,5 NM SE of IAH,5,9.260,SE,135,IAH,64E25960-75A0-495F-95BF-C92621C6B448,George Bush Intcntl/Houston,29.984435,-95.341442,KIAH
2,Aircraft reported a fix wing uas or RC aircraf...,AIRPORT,1 NM N of APA,1,1.852,N,0,APA,F45128FD-465B-4CD2-9CCA-F022A4152561,Centennial,39.570119,-104.849293,KAPA
3,Aircraft reported a blue uas 1 meter in diamet...,AIRPORT,13 NE CID,13,24.076,NE,45,CID,EAAE5D73-BBC8-464C-833B-63081F6F7FA5,The Eastern Iowa,41.884689,-91.710799,KCID
4,"Aircraft reported a uas at 2,400 feet while Nb...",AIRPORT,3 NM S of PHT,3,5.556,S,180,PHT,FDF20B1E-05CE-41A5-BF48-EB1741C983FF,Henry County,36.335944,-88.384444,KPHT
5,Aircraft reported a green and white uas 700 fe...,AIRPORT,31 W MSY,31,57.412,W,270,MSY,E00678C2-CCFF-4487-A28D-B0B2C345498B,Louis Armstrong New Orleans Intl,29.993278,-90.259028,KMSY
6,Aircraft observed a Quad-copter uas while Sbou...,AIRPORT,1 NM NW of LAS,1,1.852,NW,315,LAS,FFAE2022-BD69-475C-801F-3C80B7FCB36B,Harry Reid Intl,36.080044,-115.152235,KLAS
7,Aircraft observed a uas at the 6 o'clock posit...,AIRPORT,31 NM NNE of ATL,31,57.412,NNE,23,ATL,90091CE7-1372-4182-BA14-366919A3F134,Hartsfield - Jackson Atlanta Intl,33.636700,-84.427864,KATL
8,"MOR reported. Aircraft observed a 3 foot wide,...",AIRPORT,5 NM NW of LAS,5,9.260,NW,315,LAS,FFAE2022-BD69-475C-801F-3C80B7FCB36B,Harry Reid Intl,36.080044,-115.152235,KLAS
9,Aircraft observed a uas operation 0.5 NM W of ...,AIRPORT,0.5 NM W of TPA,0.5,0.926,W,270,TPA,94348EF7-5624-41A5-A685-7680EB5F04EB,Tampa Intl,27.975472,-82.533250,KTPA


In [14]:
def uas_lat_long(dataframe):  
    '''
    This function takes in a .csv file (string) of SkyWatch records for which the Remarks are in the Standard Format
    and uses the bearing_dist_originIdent() function to output the dataframe containing complete UAS Location information
    in terms of the UAS' Latitude and Longitude.
    
    @param dataframe : The dataframe that includes the fields 'LATITUDE', 'LONGITUDE', 'Bearing_Degrees', 
                        'Distance_Kilometers' to have UAS Lat/Long values calculated for
    
    @output new_df: returns a dataframe containing the Remark, UAS Location, FAA Airport Identifier, Distance (in NM) from 
                    the airport, and bearing abbreviation (WHERE ALL BEARINGS ARE VALID)
                
                *** it should be noted that the full dataset that includes remarks for which UAS latitudes and longitudes
                    were calculated is located in the StandardRemark_UAS_LatLong_validBearing_FAAairIdent.csv file.
                    
    
    '''
    uas_lat = []
    uas_long = []
    
    for i in range(len(dataframe)):
        lat_airport = pd.to_numeric(dataframe['LATITUDE'][i])
        long_airport = pd.to_numeric(dataframe['LONGITUDE'][i])
        b = pd.to_numeric(dataframe['Bearing_Degrees'][i])
        d = pd.to_numeric(dataframe['Distance_Kilometers'][i])

        origin = geopy.Point(lat_airport, long_airport)
        destination = geodesic(kilometers=d).destination(origin,b)

        lat2, lon2 = destination.latitude, destination.longitude

        uas_lat.append(lat2)
        uas_long.append(lon2)
        
    
    # Append UAS Lat/Long information to DataFrame
    dataframe['UAS_Latitude'] = uas_lat
    dataframe['UAS_Longitude'] = uas_long
    
    # Export final dataframe (EXCLUDING NULL BEARING INFO) to .csv file
    #dataframe.to_csv('StandardRemark_UAS_LatLong_validBearing_FAAairIdent.csv', index = False)
    
    return dataframe

In [15]:
air22_uas = uas_lat_long(air22_link)
air22_uas

,REMARKS,DATASET,UASLOCATION,Distance_NM,Distance_Kilometers,Bearing,Bearing_Degrees,IDENT,GLOBAL_ID,NAME,LATITUDE,LONGITUDE,ICAO_ID,UAS_Latitude,UAS_Longitude
0,Aircraft reported a black uas 500 FT below Air...,AIRPORT,32 NE GSO,32,59.264,NE,45,GSO,4DACDC1C-0729-4FD5-AFA4-213599F5BF4B,Piedmont Triad Intl,36.101327,-79.941123,KGSO,36.478068,-79.473509
1,Aircraft reported a grey quad copter with ligh...,AIRPORT,5 NM SE of IAH,5,9.260,SE,135,IAH,64E25960-75A0-495F-95BF-C92621C6B448,George Bush Intcntl/Houston,29.984435,-95.341442,KIAH,29.925350,-95.273631
2,Aircraft reported a fix wing uas or RC aircraf...,AIRPORT,1 NM N of APA,1,1.852,N,0,APA,F45128FD-465B-4CD2-9CCA-F022A4152561,Centennial,39.570119,-104.849293,KAPA,39.586799,-104.849293
3,Aircraft reported a blue uas 1 meter in diamet...,AIRPORT,13 NE CID,13,24.076,NE,45,CID,EAAE5D73-BBC8-464C-833B-63081F6F7FA5,The Eastern Iowa,41.884689,-91.710799,KCID,42.037777,-91.505195
4,"Aircraft reported a uas at 2,400 feet while Nb...",AIRPORT,3 NM S of PHT,3,5.556,S,180,PHT,FDF20B1E-05CE-41A5-BF48-EB1741C983FF,Henry County,36.335944,-88.384444,KPHT,36.285875,-88.384444
5,Aircraft reported a green and white uas 700 fe...,AIRPORT,31 W MSY,31,57.412,W,270,MSY,E00678C2-CCFF-4487-A28D-B0B2C345498B,Louis Armstrong New Orleans Intl,29.993278,-90.259028,KMSY,29.991934,-90.854010
6,Aircraft observed a Quad-copter uas while Sbou...,AIRPORT,1 NM NW of LAS,1,1.852,NW,315,LAS,FFAE2022-BD69-475C-801F-3C80B7FCB36B,Harry Reid Intl,36.080044,-115.152235,KLAS,36.091845,-115.166776
7,Aircraft observed a uas at the 6 o'clock posit...,AIRPORT,31 NM NNE of ATL,31,57.412,NNE,23,ATL,90091CE7-1372-4182-BA14-366919A3F134,Hartsfield - Jackson Atlanta Intl,33.636700,-84.427864,KATL,34.112912,-84.184727
8,"MOR reported. Aircraft observed a 3 foot wide,...",AIRPORT,5 NM NW of LAS,5,9.260,NW,315,LAS,FFAE2022-BD69-475C-801F-3C80B7FCB36B,Harry Reid Intl,36.080044,-115.152235,KLAS,36.139032,-115.224984
9,Aircraft observed a uas operation 0.5 NM W of ...,AIRPORT,0.5 NM W of TPA,0.5,0.926,W,270,TPA,94348EF7-5624-41A5-A685-7680EB5F04EB,Tampa Intl,27.975472,-82.533250,KTPA,27.975472,-82.542662


In [16]:
# Export to .csv 
air22_uas.to_csv('airport22OF415_uasLatLong.csv', index = False)

# AIRPORT-CEDAR

In [26]:
output25_airced = output25[output25['DATASET'] == 'AIRPORT - CEDAR'].reset_index(drop=True)
output25_airced

,index,REMARKS,UAS_Location_Runways,dateonly,timeonly,DATE,POD,PRIMARYCODE,SECONDARYCODES,REPORTINGFACILITY,...,UASDIM.CEDAR,UASFWROTOR.CEDAR,PILOTREPORTEDNMAC.CEDAR,TCASRA.CEDAR,SUMMARY.CEDAR,QAFINDINGS.CEDAR,RWYLOCATION,UASLOCATION,DATASET,cityname
0,1491,Aircraft observed a uas while circling overhea...,UNKN,7/23/2021,21:42,2021-07-23T21:42Z,DEN,UAS,NaN,CLE,...,UAS Dimensions (feet): NA,Fixed Wing,No,No,N698DA WAS CLIMBING FOR PARACHUE OPERATIONS OV...,QA reviewed. Not reported as NMAC.,NaN,80 NM WIT,AIRPORT - CEDAR,NaN


In [27]:
# Export to .csv 
output25_airced.to_csv('airCed01OF33_EXCEPTIONS.csv', index = False)

In [17]:
output25_airced = output25_df[output25_df['DATASET'] == 'AIRPORT - CEDAR'].drop(columns = ['cityname']).reset_index(drop=True)
output25_airced

,REMARKS,DATASET,UASLOCATION
0,Aircraft observed a uas while circling overhea...,AIRPORT - CEDAR,80 NM WIT


In [18]:
air1 = uas_loc_parts(output25_airced)
air1

,REMARKS,DATASET,UASLOCATION,Distance_NM,Distance_Kilometers,Bearing,Bearing_Degrees,IDENT
0,Aircraft observed a uas while circling overhea...,AIRPORT - CEDAR,80 NM WIT,80,148.16,80,80,WIT


In [19]:
air1_link = link_airportsCleaned(air1)
air1_link

,REMARKS,DATASET,UASLOCATION,Distance_NM,Distance_Kilometers,Bearing,Bearing_Degrees,IDENT,GLOBAL_ID,NAME,LATITUDE,LONGITUDE,ICAO_ID
0,Aircraft observed a uas while circling overhea...,AIRPORT - CEDAR,80 NM WIT,80,148.16,80,80,WIT,NaN,NaN,NaN,NaN,NaN


# CITY

In [28]:
output25_city = output25[output25['DATASET'] == 'CITY'].reset_index(drop=True)
output25_city

,index,REMARKS,UAS_Location_Runways,dateonly,timeonly,DATE,POD,PRIMARYCODE,SECONDARYCODES,REPORTINGFACILITY,...,UASDIM.CEDAR,UASFWROTOR.CEDAR,PILOTREPORTEDNMAC.CEDAR,TCASRA.CEDAR,SUMMARY.CEDAR,QAFINDINGS.CEDAR,RWYLOCATION,UASLOCATION,DATASET,cityname
0,69,"Aircraft observed a quadcopter type uas, 200 f...",UNKN,7/10/2018,2:44,2018-07-10T02:44Z,DEN,UAS,NaN,I90,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10 NM N,CITY,Houston


In [29]:
# Export to .csv 
output25_city.to_csv('city01OF33_EXCEPTIONS.csv', index = False)

# NAVAID

In [20]:
output25_navaid = output25_df[output25_df['DATASET'] == 'NAVAID'].drop(columns = ['cityname']).reset_index(drop=True)
output25_navaid

,REMARKS,DATASET,UASLOCATION
0,"Aircraft reported a uas 1,000 feet low while S...",NAVAID,8 NM W of TRV


In [21]:
nav1 = uas_loc_parts(output25_navaid)
nav1

,REMARKS,DATASET,UASLOCATION,Distance_NM,Distance_Kilometers,Bearing,Bearing_Degrees,IDENT
0,"Aircraft reported a uas 1,000 feet low while S...",NAVAID,8 NM W of TRV,8,14.816,W,270,TRV


In [22]:
def nav_link (dataframe):
    '''
    This functions merges the dataframe input by the user with the cleaned NAVAID_cleaned.csv dataset
    '''
    
    # Read in Cleaned NAVAID Systems dataset
    nav_clean = pd.read_csv('C:/Users/grace/OneDrive/Desktop/GMU/DAEN690/NAVAID_cleaned.csv')
    
    output_df = pd.merge(dataframe, nav_clean, on = 'IDENT', how = 'left').reset_index(drop=True)
    
    return output_df

In [23]:
nav1_link = nav_link(nav1)
nav1_link

,REMARKS,DATASET,UASLOCATION,Distance_NM,Distance_Kilometers,Bearing,Bearing_Degrees,IDENT,LONGITUDE,LATITUDE,NAME_TXT,CITY
0,"Aircraft reported a uas 1,000 feet low while S...",NAVAID,8 NM W of TRV,8,14.816,W,270,TRV,-80.489728,27.678429,TREASURE,VERO BEACH


In [24]:
nav1_uas = uas_lat_long(nav1_link)
nav1_uas

,REMARKS,DATASET,UASLOCATION,Distance_NM,Distance_Kilometers,Bearing,Bearing_Degrees,IDENT,LONGITUDE,LATITUDE,NAME_TXT,CITY,UAS_Latitude,UAS_Longitude
0,"Aircraft reported a uas 1,000 feet low while S...",NAVAID,8 NM W of TRV,8,14.816,W,270,TRV,-80.489728,27.678429,TREASURE,VERO BEACH,27.678348,-80.639912


In [25]:
# Export to .csv 
nav1_uas.to_csv('navaid01OF415_uasLatLong.csv', index = False)